## > Import libraries

In [1]:
import time
from dotenv import load_dotenv
import os
import sys
import subprocess
import shutil
from jsmin import jsmin
import re 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import git  # pip install gitpython

## > Configs 

#### User configs

In [2]:
URL_ISSUE = "https://github.com/fictadvisor/fictadvisor-web/issues/864"
IS_FRONTEND = True
RE_CLONE_REPOSITORY = False

#### Constant configs

In [3]:
CONTACT_NAME = "https://t.me/akiyama_kiyoshi"
URL_FRONTEND_REPO = "https://github.com/fictadvisor/fictadvisor-web.git" 
DIRECTORY_FRONTEND_REPO = '../../../assets/repositories/frontend/'
OPENAI_API_KEY=""

## > Utils 

In [4]:
def is_github_issue(url):
    pattern = r'https://github\.com/[^/]+/[^/]+/issues/\d+'
    return bool(re.match(pattern, url))

def issue_meta(url, driver, to_close = True):
    """ 
    [url] - issue URL 
    Returns: {"title": ?, "description": ?} 
    """
    if not is_github_issue(url):
        example_url = 'https://github.com/fictadvisor/fictadvisor-web/issues/655'
        raise Exception(f" Wrong URL \n Your url: {url} \n Exampple url: {example_url} ")

    try: 
        driver.get(url)
        issue_title = driver.find_element(By.CLASS_NAME, "js-issue-title").text
        issue_description = driver.find_elements_by_css_selector('d-block,comment-body,markdown-body,js-comment-body')
        if to_close: 
            driver.quit()
        
        return {"title": issue_title, "description":issue_description}
    except: 
        driver.quit()
        raise Exception(f" Wrong URL \n Your url: {url} \n Something gone wrong during opening ")
    
def get_file_tree(path):
    file_tree = ""
    for dirpath, dirnames, filenames in os.walk(path):
        for file_name in filenames:
            file_tree += f"\t{file_name}\n"
    return file_tree

def get_all_folder_names(path):
    """ removes duplicates """
    folders = []
    for entry in os.scandir(path):
        if entry.is_dir():
            folders.append(os.path.basename(entry.path))
            folders.extend(get_all_folder_names(entry.path))
    return list(set(folders))

def find_folders(base_path, folder_names):
    """ returns paths to folder names """
    folder_paths = {}
    for root, dirs, files in os.walk(base_path):
        for folder_name in folder_names:
            if folder_name in dirs:
                folder_paths[folder_name] = os.path.join(root, folder_name)
    return folder_paths

def get_all_contents(path):
    contents = []
    for dirpath, dirnames, filenames in os.walk(path):
        for dirname in dirnames:
            contents.append(os.path.join(dirpath, dirname))
        for filename in filenames:
            contents.append(os.path.join(dirpath, filename))
    return contents

def minify_js_file(file_path):
    with open(file_path, 'r') as js_file:
        return jsmin(js_file.read())

## > Setup

In [5]:
if not IS_FRONTEND: raise Exception(" Backend repository is not supported right now, for more details contact " + CONTACT_NAME)

In [6]:
driver = webdriver.Chrome(ChromeDriverManager().install())


/bin/sh: line 1: google-chrome: command not found
/bin/sh: line 1: google-chrome: command not found


load env file 

In [7]:
load_dotenv('../../.env')

True

## > Getting repository

In [8]:
if RE_CLONE_REPOSITORY:     
    directory = '../assets/repositories/frontend/'
    if os.path.exists(directory):
        shutil.rmtree(directory)

    subprocess.run(["git", "clone", URL_FRONTEND_REPO, DIRECTORY_FRONTEND_REPO])

## > Setup

## Prompting 
### Steps 
1. GPT should

In [9]:
from openai import OpenAI
client = OpenAI()

In [10]:
folder_names_joined=','.join(get_all_folder_names(DIRECTORY_FRONTEND_REPO))
issue = issue_meta(URL_ISSUE, driver, )

In [11]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Act as experienced assistant. You are going to help users with their tasks. In this answer You need to answer what folders do you need to see to better answer the question. provide only folder names. Provide only folder names separated with \n. Do  not write anything else"},
    {"role": "user", "content": f" FolderStructure: {folder_names_joined}; \n What folders do you need to check to better answer this:  {issue['title']} \n {issue['description']}"}
  ]
)

In [12]:
folder_names = completion.choices[0].message.content.split(",")  
folder_paths = find_folders('../', folder_names)
structure_prompt = "file structure of needed folders: "

for name, path in folder_paths.items():
  structure_prompt += f"{name}\n {get_all_contents(path)}\n"  
  
  print(structure_prompt)

In [13]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Act as experienced assistant. You are going to help users with their tasks. In this answer You need to answer what files do you need to see to better answer the question. provide only file names. Provide only file names separated with \n. Do  not write anything else"},
    {"role": "user", "content": f" FolderStructure: {structure_prompt}; \n What files do you need to check to better answer this:  {issue['title']}"}
  ]
)

In [14]:
paths = structure_prompt
structure_prompt = "" 

""" TODO: SELECT NEEDED FILES LATER ( INSTEAD OF MINIFYING THEM ALL ) """
""" TODO: HANDLE ERRORS WHEN FILE READING """
for js_file_path in paths:
    try: 
        minified = minify_js_file(js_file_path)
        print(minified)
        structure_prompt += (f"file: {js_file_path}, content: {minified}")
    except: 
        pass
    

In [15]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "you have been provided with the files you need to respond. If there is not enough information, try to answer anyway, but ask for more context if necessary. You need to make a detailed plan of how to complete the task, what to look for, and what can go wrong, and write the code for it."},
    {"role": "user", "content": f" How do i do this task: {issue['title']} using this: {structure_prompt}. Assume that in project used MUI, React, Next, Typescript"}
  ]
)
plan = completion.choices[0].message.content

In [16]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Act as high experienced React developer. You need to write code that asked using plan provided if needed "},
    {"role": "user", "content": f" How do i do this task: {issue['title']} using this: {structure_prompt}. Assume that in project used MUI, React, Next, Typescript\n plan: {plan}" }
  ]
)
print(completion.choices[0].message.content)

""" TODO: NEED MORE CONTEXT FROM COMPONENTS, MAKE LIST OF THEM AND FEED TO GPT WHEN NEEDED """

Assuming you are using React and MUI in your project, here's a possible implementation to fix the bug mentioned:

Step 1: Identify the code responsible for validating the password field.

Locate the code that performs the password field validation. This might be in the component that corresponds to the "Безпека" tab or any related utility functions.

Step 2: Check the existing validation logic.

Check if the existing validation logic handles Cyrillic characters correctly. Look for any regex patterns or validation functions being used.

Step 3: Update the validation logic.

If the existing validation logic does not support Cyrillic characters, update it with the following approach:

```typescript
import React, { useState } from "react";
import TextField from "@mui/material/TextField";

const MyComponent = () => {
  const [password, setPassword] = useState("");
  const [isValid, setIsValid] = useState(true);

  const handlePasswordChange = (event: React.ChangeEvent<HTMLInputElement>) => 

' TODO: NEED MORE CONTEXT FROM COMPONENTS, MAKE LIST OF THEM AND FEED TO GPT WHEN NEEDED '